# Exercise 2 - shell, pipes, and csvkit

In this exercise, we'll review a few shell commands and explore working with pipes and csvkit.

You will need to fill in commands whenever prompted.  Please replace the text with your solution.

Remember to submit your completed `.ipynb` file to Blackboard and to add/commit it to your Git repository and push it to GitHub.


## Part 1 - shell commands, redirection, and pipes

### Basic shell commands and redirection

Create a directory called `part1` using `mkdir`.

In [1]:
!mkdir part1

Rename `part1` to `partone` using `mv`.

In [2]:
!mv part1 partone
!ls

exercise-02.ipynb  partone  pg11.txt  pg2500.txt


Create a file named `filelist.txt` using the output from `ls` and the output redirector `>`.

In [3]:
!ls > filelist.txt

In [4]:
!cat filelist.txt

exercise-02.ipynb
filelist.txt
partone
pg11.txt
pg2500.txt


Append to `filelist.txt` using the output appending redirector `>>`.  Note the difference between the single `>` and double `>>`.

In [5]:
!ls >> filelist.txt
!cat filelist.txt

exercise-02.ipynb
filelist.txt
partone
pg11.txt
pg2500.txt
exercise-02.ipynb
filelist.txt
partone
pg11.txt
pg2500.txt


In [6]:
!ls > filelist.txt
!cat filelist.txt

exercise-02.ipynb
filelist.txt
partone
pg11.txt
pg2500.txt


The redirector '>' overwrites the end location with the output that is being redirected. The redirector '>>' appends the output to the end location without overwriting.

### Your turn

Complete the following tasks in the cells provided.  All the tests in the testing cells (with the `assert` statements) below should pass without error - be sure to execute those as well, and if you see errors, fix your answer and try testing again until there are no errors.

Create a directory called `mydirectory`.

In [7]:
# YOUR CODE HERE
!mkdir mydirectory

#raise NotImplementedError()

In [8]:
import os
assert 'mydirectory' in os.listdir('.')

Using `ls` and output redirection, create a file called `myfiles.txt` in the directory `mydirectory` that contains the list of files in the current directory.

In [9]:
# YOUR CODE HERE
!ls > mydirectory/myfiles.txt

#raise NotImplementedError()

In [10]:
assert 'myfiles.txt' in os.listdir('mydirectory')

In [11]:
myfiles = open('mydirectory/myfiles.txt').read()
assert 'exercise-02.ipynb' in myfiles

Clean up the directory you just created by removing its contents (the file you created) using `rm`.

In [12]:
rm mydirectory/myfiles.txt

In [13]:
assert 'myfiles.txt' not in os.listdir('mydirectory')

Now remove the directory itself using `rmdir`.

In [14]:
# YOUR CODE HERE
!rmdir mydirectory

#raise NotImplementedError()

In [15]:
assert 'mydirectory' not in os.listdir('.')

### Filters and pipes

Let's look at something a little more interesting.  Download the text of Herman Hesse's *Siddhartha* from [Project Gutenberg](http://www.gutenberg.org/):

In [16]:
!wget https://www.gutenberg.org/cache/epub/2500/pg2500.txt

--2016-09-08 20:46:48--  https://www.gutenberg.org/cache/epub/2500/pg2500.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2016-09-08 20:46:48 ERROR 403: Forbidden.



*Note*: sometimes Project Gutenberg restricts access.  If that creates an error for you, you should be able to `wget` the same file from our class repository on GitHub at the url https://github.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/raw/master/exercises/pg2500.txt.

However you get the file, let's rename it to something easier to remember.

In [17]:
!mv pg2500.txt siddhartha.txt

`head` and `tail` are very useful.  They let you take a quick peek at the start and end of files.

In [18]:
!head siddhartha.txt

In [19]:
!tail siddhartha.txt

`grep` is one of the most useful filters.  It lets you search for and match lines that contain specific expressions.  For example, to find mentions of "copyright":

In [20]:
!grep copyright siddhartha.txt

Notice anything that those lines have in common?

They all have the word "copyright" in them.

Let's add a little more information by including the `-n` flag to add matching line numbers.

In [21]:
!grep -n copyright siddhartha.txt

Now let's look for any mention of "river".  This will match a lot of text, so we'll just take the first 10 matching lines by *piping* the output from `grep` into `head`.

In [22]:
!grep -n river siddhartha.txt | head











grep: write error: Broken pipe


How many lines contain "river"?  We can count by piping into the word count tool `wc`.

In [23]:
!grep river siddhartha.txt | wc

    109    1365    7443


That's 109 matching lines, containing 1365 words and 7979 characters.  If you just wanted the lines by themselves, use `wc -l`:

In [24]:
!grep river siddhartha.txt | wc -l

109


What if we want to match both upper- and lower-case text?  Use `grep -i`:

In [25]:
!grep time siddhartha.txt | wc -l

166


In [26]:
!grep -i time siddhartha.txt | wc -l

167


### Your turn

How many lines in *Siddhartha* contain "other" (just lower-case)?  Start by using `grep` to extract lines that match the word "other" in `siddhartha.txt` and redirecting it to a file called `other-lines.txt`.

In [27]:
# YOUR CODE HERE
!grep other siddhartha.txt > other-lines.txt

# raise NotImplementedError()

In [28]:
%sc h_other = head -1 other-lines.txt
assert "other" in h_other

In [29]:
%sc t_other = tail -1 other-lines.txt
assert "other" in t_other

Now count up the lines in the file you created using `wc`.

In [30]:
# YOUR CODE HERE
!wc -l < other-lines.txt

#raise NotImplementedError()

127


In [31]:
# ERROR IN GRADER CODE

#assert "127" in _

Your answer should be 127!

### Counting words with `grep`

By piping commands together we can do a lot of powerful things right at the command line.  Let's create a count of the most commonly occurring words in *Siddhartha*.  To do that, we could write a Python script that just counts words, but with the command line shell tools we only need to put a proper pipeline together and we can often accomplish tasks like this in one line.

First we need to split up the text lines into a word per line.  There are `grep` flags for that!

In [32]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | head -10

The
Project
Gutenberg
EBook
of
Siddhartha
by
Herman
Hesse
This
grep: write error
cat: write error: Broken pipe


Now we need to sort them and count the unique tokens.  `sort` solves the first problem.

In [33]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | sort | head -10

000
1500
1887
20
2001
2008
2011
2013
23
2500
sort: write failed: standard output: Broken pipe
sort: write error


And `uniq -c` solves the second problem.

In [34]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | sort | uniq -c | head -25

      1 000
      1 1500
      1 1887
      1 20
      1 2001
      1 2008
      1 2011
      1 2013
      1 23
      4 2500
      1 30
      1 4557
      1 50
      2 501
      1 596
      1 60
      1 6221541
      1 64
      1 801
      1 809
      1 84116
      2 90
      1 99712
      2 abandon
      2 abandoned
uniq: write error: Broken pipe


But there's a catch... do you see it?

We need to convert all the words down into lower case so that we are correctly counting unique words.  There's another command, `tr`, for that.

In [35]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | head -25

      1 000
      1 1500
      1 1887
      1 20
      1 2001
      1 2008
      1 2011
      1 2013
      1 23
      4 2500
      1 30
      1 4557
      1 50
      2 501
      1 596
      1 60
      1 6221541
      1 64
      1 801
      1 809
      1 84116
      2 90
      1 99712
      2 abandon
      2 abandoned
uniq: write error: Broken pipe


...and if we want to know only the top 10 words in Siddhartha, we need to sort the output.

In [36]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | sort | head -10

      1 000
    101 do
    106 more
    108 any
    108 so
     10 access
     10 agree
     10 arms
     10 ask
     10 ate
sort: write failed: standard output: Broken pipe
sort: write error


But that sorts by character, not number.  Fortunately, `sort -n` does what we want.

In [37]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | sort -n | head -10

      1 000
      1 1500
      1 1887
      1 20
      1 2001
      1 2008
      1 2011
      1 2013
      1 23
      1 30
sort: write failed: standard output: Broken pipe
sort: write error


But that's the wrong end of the list!  Two ways to fix that:  (a) use `tail` instead of `head`; (b) use `sort -rn`, which will sort in reverse order.  Let's try the latter.

In [38]:
!cat siddhartha.txt | grep -oE '\w{{2,}}' | tr '[:upper:]' '[:lower:]' | sort | uniq -c | sort -rn | head -10

   2221 the
   1434 and
   1225 to
   1106 of
    960 he
    708 his
    686 in
    540 you
    524 had
    512 was
sort: write failed: standard output: Broken pipe
sort: write error


### Your turn

Download *Alice in Wonderland* from http://www.gutenberg.org/cache/epub/11/pg11.txt (or https://github.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/raw/master/exercises/pg11.txt if the first url doesn't work).

In [39]:
# YOUR CODE HERE
!wget https://github.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/raw/master/exercises/pg11.txt

#raise NotImplementedError()

--2016-09-08 20:47:05--  https://github.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/raw/master/exercises/pg11.txt
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/pg11.txt [following]
--2016-09-08 20:47:05--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/pg11.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.20.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.20.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167518 (164K) [text/plain]
Saving to: ‘pg11.txt.1’

pg11.txt.1          100%[=====================>] 163.59K  --.-KB/s   in 0.02s  

2016-09-08 20:47:05 (7.92 MB/s) - ‘pg11.txt.1’ saved [167518/167518]



In [40]:
!ls

exercise-02.ipynb  other-lines.txt  pg11.txt	siddhartha.txt
filelist.txt	   partone	    pg11.txt.1


In [41]:
assert 'pg11.txt' in os.listdir('.')

Now rename `pg11.txt` to `alice.txt`.

In [42]:
# YOUR CODE HERE
!mv pg11.txt alice.txt

#raise NotImplementedError()

In [43]:
assert 'alice.txt' in os.listdir('.')

Take a look at the next cell.  Will it find the top 25 unique words in *Alice in Wonderland* successfully?

In [44]:
!cat alice.txt | grep -oE '\w{{2,}}' | sort | uniq -c | head -25

      1 000
      4 11
      1 1500
      1 1887
      1 1994
      2 20
      1 2001
      1 2008
      1 2011
      1 25
      1 30
      1 4557
      1 50
      2 501
      1 596
      1 60
      1 6221541
      1 64
      1 801
      1 809
      1 84116
      2 90
      1 99712
      2 abide
      1 able
uniq: write error: Broken pipe


Describe what needs to be done to the previous cell to get it to work correctly.  Describe it using words, explaining the issues, rather than using shell commands!

The previous command differentiates between upper and lower-case versions of the same word, while we would not to treat both versions of the same word as one unique word. To correct this, we can upper-case characters to lower-case. This command also sorts based on alphanumeric character order rather than word count. Once the list is sorted by word count, we also need to make sure that we're sorting it in the order that shows the most common words at the top of the list.

Okay, now implement your solution using shell commands with a pipeline.

In [46]:
# YOUR CODE HERE
!cat alice.txt | grep -oE '\w{{2,}}'| tr '[:upper:]' '[:lower:]' | sort | uniq -c | sort -rn | head -25

#raise NotImplementedError()

   1818 the
    940 and
    809 to
    631 of
    610 it
    553 she
    481 you
    462 said
    431 in
    403 alice
    358 was
    330 that
    274 as
    248 her
    228 with
    227 at
    204 on
    200 all
    181 this
    179 for
    178 had
    175 but
    167 be
    166 not
    155 they
sort: write failed: standard output: Broken pipe
sort: write error


## Part 2 - csvkit basics

Let's look at some CSV data using csvkit.  Download the 2015 fourth quarter trip dataset from [Capital Bikeshare's trip history data](https://www.capitalbikeshare.com/trip-history-data):

In [47]:
!wget https://s3.amazonaws.com/capitalbikeshare-data/2015-Q4-cabi-trip-history-data.zip

--2016-09-08 20:56:47--  https://s3.amazonaws.com/capitalbikeshare-data/2015-Q4-cabi-trip-history-data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.40.26
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.40.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14308794 (14M) [application/zip]
Saving to: ‘2015-Q4-cabi-trip-history-data.zip’

2015-Q4-cabi-trip-h 100%[=====================>]  13.65M  28.6MB/s   in 0.5s   

2016-09-08 20:56:48 (28.6 MB/s) - ‘2015-Q4-cabi-trip-history-data.zip’ saved [14308794/14308794]



Let's unzip it, rename it to something short, and take a look.

In [ ]:
!unzip 2015-Q4-cabi-trip-history-data.zip

In [ ]:
!mv 2015-Q4-Trips-History-Data.csv q4.csv

In [ ]:
!head q4.csv

csvkit gives us great tools for examining and working with CSV data.  We start by looking at the columns:

In [ ]:
!csvcut -n q4.csv

We can also extract just a few columns with `csvcut`:

In [ ]:
!csvcut -c1,5,7 q4.csv | head -10

...and make it look better with `csvlook`:

In [ ]:
!csvcut -c1,5,7 q4.csv | head -10 | csvlook

It gets even better.  Try `csvgrep`:

In [ ]:
!csvcut -c1,5,7 q4.csv | csvgrep -c3 -m '21st & I St NW' | head -10 | csvlook

But wait, there's more:

In [ ]:
!csvcut -c1,5,7 q4.csv | csvgrep -c3 -m '21st & I St NW' | csvsort -c2 | head -10 | csvlook

And you can perform basic statistics very easily:

In [ ]:
!csvcut -c1,5,7 q4.csv | csvgrep -c3 -m '21st & I St NW' | csvcut -c1 | csvstat

### Your turn

Which set of trips had the longer average trip duration:  trips *starting* at "Massachusetts Ave & Dupont Circle NW", or trips *ending* at "Massachusetts Ave & Dupont Circle NW"?

Use as many new cells as you need to compute the answer, and then write in your answer below in the "YOUR ANSWER HERE" cell.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()